## 데이터 불러오기

In [1]:
from geoband.API import *
import os
from glob import glob
import json
import pandas as pd
import numpy as np
import folium
from folium import plugins
import geopandas as gpd

In [2]:
df1 = pd.read_csv('1.오산시_주정차단속(2018~2020).csv')
df2 = json.load(open('2.오산시_어린이교통사고_격자.geojson', encoding = 'utf-8'))
df3 = json.load(open('3.오산시_차량등록현황_격자.geojson', encoding = 'utf-8'))
df4 = json.load(open('4.오산시_연령별_거주인구격자(총인구).geojson', encoding = 'utf-8'))
df5 = json.load(open('5.오산시_연령별_거주인구격자(유소년).geojson', encoding = 'utf-8'))
df6 = json.load(open('6.오산시_연령별_거주인구격자(생산가능인구).geojson', encoding = 'utf-8'))
df7 = json.load(open('7.오산시_연령별_거주인구격자(고령).geojson', encoding = 'utf-8'))
df8 = pd.read_csv('8.오산시_유동인구(2019).csv')
df9 = pd.read_csv('9.오산시_어린이보호구역.csv')
df10 = pd.read_csv('10.오산시_학교위치정보.csv')
df11 = json.load(open('11.오산시_초등학교_통학구.geojson', encoding = 'utf-8'))
df12 = json.load(open('12.오산시_중학교_학군.geojson', encoding = 'utf-8'))
df13 = pd.read_csv('13.오산시_어린이집_유치원현황.csv')
df14 = pd.read_csv('14.오산시_기상데이터(2010~2019).csv')
df15 = pd.read_csv('15.오산시_무인교통단속카메라.csv')
df16 = pd.read_csv('16.오산시_도로안전표지표준데이터.csv')
df17 = json.load(open('17.오산시_횡단보도.geojson', encoding = 'utf-8'))
df18 = pd.read_csv('18.오산시_과속방지턱표준데이터.csv')
df19 = json.load(open('19.오산시_신호등.geojson', encoding = 'utf-8'))
df20 = pd.read_csv('20.오산시_CCTV설치현황.csv')
df21 = json.load(open('21.오산시_인도.geojson', encoding = 'utf-8'))
df22 = pd.read_csv('22.오산시_버스정류장.csv')
df23 = json.load(open('23.오산시_상세도로망_LV6.geojson', encoding = 'utf-8'))
df24 = pd.read_csv('24.평일_전일,시간대별_오산시_추정교통량_Level6.csv')
df25 = pd.read_csv('25.평일_전일_오산시_혼잡빈도강도_Level6.csv')
df26 = pd.read_csv('26.평일_전일_오산시_혼잡시간강도_Level6.csv')
df27 = json.load(open('27.오산시_도로명주소_건물.geojson', encoding = 'utf-8'))
df28 = json.load(open('28.오산시_건물연면적_격자.geojson', encoding = 'utf-8'))
df29 = pd.read_csv('29.오산시_체육시설현황.csv')
df30 = pd.read_csv('30.오산시_학원_및_교습소_현황.csv')
df31 = json.load(open('31.오산시_법정경계(시군구).geojson', encoding = 'utf-8'))
df32 = json.load(open('32.오산시_행정경계(읍면동).geojson', encoding = 'utf-8'))
df33 = json.load(open('33.오산시_법정경계(읍면동).geojson', encoding = 'utf-8'))
df34 = json.load(open('34.오산시_지적도.geojson', encoding = 'utf-8'))

/opt/app-root/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 데이터 전처리

### 1. 격자(Polygon)형 데이터 전처리
- 100mx100m 격자의 중심점을 도출
- 중심점이 격자 내 좌표에 속해있다면, 격자에 속한 것으로 Count
- 활용한 Feature: 사고횟수, 차량개수, 유소년인구수, 생산가능인구수, 고령인구수, 건물연면적

### 2. 좌표형 데이터 전처리
- 100mx100m 격자의 위도의 최소/최대 및 경도의 최소/최대 범위를 구함
- 해당 격자내에 좌표가 속해있다면, 격자에 속한 것으로 Count
- 활용한 Feature: 주정차단속건수, 유동인구, 어린이보호구역개수, 어린이보호구역CCTV개수, 초등학교개수, 유치원/어린이집개수, 무인교통단속카메라개수, 도로안전표지판개수, 횡단보도개수, 과속방지턱개수, 신호등개수, 버스정류장개수, 건물개수, 체육시설개수, 학원교습소개수

### 3. MultiLineString형 데이터 전처리
- Buffer를 활용해 Polygon형태로 변환
- 격자의 Polygon과 Buffer의 Polygon이 겹치는(intersection)부분이 있다면 해당 격자에 속한 것으로 Count
- 활용한 Feature: 전체 추정교통량, 혼잡빈도강도, 혼잡시간강도

## 1. 격자(Polygon)형 데이터 전처리

### df2: 사고횟수

In [3]:
# 2. 오산시 어린이 교통사고 격자

df2_1 = pd.DataFrame(columns=['격자명','사고횟수'])
for i in range(len(df2['features'])):
    a = df2['features'][i]['properties']['gid']
    b = df2['features'][i]['properties']['accident_cnt']
    c = np.mean(np.array(df2['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df2['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '사고횟수':b, '경도': c,'위도': d}
    df2_1 = df2_1.append(new_data, ignore_index=True)

df2_1['사고횟수'] = df2_1['사고횟수'].astype('int32')
df2_1

,격자명,사고횟수,경도,위도
0,다사551085,0,126.994776,37.174635
1,다사551086,0,126.994770,37.175537
2,다사551087,0,126.994764,37.176438
3,다사551088,0,126.994758,37.177340
4,다사552085,0,126.995902,37.174640
...,...,...,...,...
4505,다사646052,0,127.101944,37.145297
4506,다사646053,0,127.101940,37.146199
4507,다사647050,0,127.103080,37.143498
4508,다사647051,0,127.103075,37.144400


### df3:  차량개수

In [4]:
# 3. 오산시 차량등록현황 격자

df3_1 = pd.DataFrame(columns=['격자명','차량개수'])
for i in range(len(df3['features'])):
    a = df3['features'][i]['properties']['gid']
    b = df3['features'][i]['properties']['car_cnt']
    c = np.mean(np.array(df3['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df3['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '차량개수':b, '경도': c,'위도': d}
    df3_1 = df3_1.append(new_data, ignore_index=True)

df3_1['차량개수'] = df3_1['차량개수'].astype('int32')
df3_1

,격자명,차량개수,경도,위도
0,다사551085,0,126.994776,37.174635
1,다사551086,0,126.994770,37.175537
2,다사551087,0,126.994764,37.176438
3,다사551088,0,126.994758,37.177340
4,다사552085,0,126.995902,37.174640
...,...,...,...,...
4505,다사646052,0,127.101944,37.145297
4506,다사646053,0,127.101940,37.146199
4507,다사647050,0,127.103080,37.143498
4508,다사647051,0,127.103075,37.144400


### df5: 유소년인구수

In [5]:
# 5. 오산시 연령별 거주인구 격자(유소년)

df5_1 = pd.DataFrame(columns=['격자명','유소년인구수'])
for i in range(len(df5['features'])):
    a = df5['features'][i]['properties']['gid']
    b = df5['features'][i]['properties']['val']
    c = np.mean(np.array(df5['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df5['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '유소년인구수':b, '경도': c,'위도': d}
    df5_1 = df5_1.append(new_data, ignore_index=True)

df5_1['유소년인구수'] = df5_1['유소년인구수'].astype('float64')
df5_1 = df5_1.fillna(0)
df5_1

,격자명,유소년인구수,경도,위도
0,다사551085,0.0,126.994776,37.174635
1,다사551086,0.0,126.994770,37.175537
2,다사551087,0.0,126.994764,37.176438
3,다사551088,0.0,126.994758,37.177340
4,다사552085,0.0,126.995902,37.174640
...,...,...,...,...
4505,다사646052,0.0,127.101944,37.145297
4506,다사646053,0.0,127.101940,37.146199
4507,다사647050,0.0,127.103080,37.143498
4508,다사647051,0.0,127.103075,37.144400


### df6: 생산가능인구수

In [6]:
# 6. 오산시 연령별 거주인구 격자(생산가능인구)

df6_1 = pd.DataFrame(columns=['격자명','생산가능인구수'])
for i in range(len(df6['features'])):
    a = df6['features'][i]['properties']['gid']
    b = df6['features'][i]['properties']['val']
    c = np.mean(np.array(df6['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df6['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '생산가능인구수':b, '경도': c,'위도': d}
    df6_1 = df6_1.append(new_data, ignore_index=True)

df6_1['생산가능인구수'] = df6_1['생산가능인구수'].astype('float64')
df6_1 = df6_1.fillna(0)
df6_1

,격자명,생산가능인구수,경도,위도
0,다사551085,0.0,126.994776,37.174635
1,다사551086,0.0,126.994770,37.175537
2,다사551087,0.0,126.994764,37.176438
3,다사551088,0.0,126.994758,37.177340
4,다사552085,0.0,126.995902,37.174640
...,...,...,...,...
4505,다사646052,0.0,127.101944,37.145297
4506,다사646053,0.0,127.101940,37.146199
4507,다사647050,0.0,127.103080,37.143498
4508,다사647051,0.0,127.103075,37.144400


### df7: 고령인구수

In [7]:
# 7. 오산시 연령별 거주인구 격자(고령)

df7_1 = pd.DataFrame(columns=['격자명','고령인구수'])
for i in range(len(df7['features'])):
    a = df7['features'][i]['properties']['gid']
    b = df7['features'][i]['properties']['val']
    c = np.mean(np.array(df7['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df7['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '고령인구수':b, '경도': c,'위도': d}
    df7_1 = df7_1.append(new_data, ignore_index=True)

df7_1['고령인구수'] = df7_1['고령인구수'].astype('float64')
df7_1 = df7_1.fillna(0)
df7_1

,격자명,고령인구수,경도,위도
0,다사551085,0.0,126.994776,37.174635
1,다사551086,0.0,126.994770,37.175537
2,다사551087,0.0,126.994764,37.176438
3,다사551088,0.0,126.994758,37.177340
4,다사552085,0.0,126.995902,37.174640
...,...,...,...,...
4505,다사646052,0.0,127.101944,37.145297
4506,다사646053,0.0,127.101940,37.146199
4507,다사647050,0.0,127.103080,37.143498
4508,다사647051,0.0,127.103075,37.144400


### df28: 건물연면적

In [8]:
# 28. 오산시 건물 연면적 격자

df28_1 = pd.DataFrame(columns=['격자명','건물연면적'])
for i in range(len(df28['features'])):
    a = df28['features'][i]['properties']['gid']
    b = df28['features'][i]['properties']['val']
    c = np.mean(np.array(df28['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df28['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '건물연면적':b, '경도': c,'위도': d}
    df28_1 = df28_1.append(new_data, ignore_index=True)

df28_1['건물연면적'] = df28_1['건물연면적'].astype('float64')
df28_1 = df28_1.fillna(0)
df28_1

,격자명,건물연면적,경도,위도
0,다사551085,0.0,126.994776,37.174635
1,다사551086,0.0,126.994770,37.175537
2,다사551087,0.0,126.994764,37.176438
3,다사551088,0.0,126.994758,37.177340
4,다사552085,0.0,126.995902,37.174640
...,...,...,...,...
4505,다사646052,0.0,127.101944,37.145297
4506,다사646053,0.0,127.101940,37.146199
4507,다사647050,0.0,127.103080,37.143498
4508,다사647051,0.0,127.103075,37.144400


## 2. 좌표형 데이터 전처리

### df1: 주정차단속건수

In [9]:
# 1. 오산시_주정차단속(2018~2020)

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

df1_1 = pd.DataFrame({'격자명':list_tmp})
df1_1['주정차단속건수'] = 0


for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df1)):
        if (df1.loc[j, '단속위치_위도'] >= min_lat) & (df1.loc[j, '단속위치_위도'] <= max_lat) & (df1.loc[j, '단속위치_경도'] >= min_lon) & (df1.loc[j, '단속위치_경도'] <= max_lon):
            df1_1.loc[i,'주정차단속건수'] = df1_1.loc[i,'주정차단속건수']+1

df1_1

,격자명,주정차단속건수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


### df8: 유동인구

In [10]:
# 어린이가 많이 다니는 14시 ~ 20시로 한정
df8_1 = df8.loc[:,['lon','lat','TMST_14','TMST_15','TMST_16','TMST_17','TMST_18','TMST_19','TMST_20']]
df8_1['mean'] = (df8_1['TMST_14'] + df8_1['TMST_15'] + df8_1['TMST_16'] + df8_1['TMST_17'] + df8_1['TMST_18'] + df8_1['TMST_19'] +df8_1['TMST_20'])/7
df8_1 = df8_1.loc[:,['lon','lat','mean']]

# 위도와 경도기준 groupby
df8_2 = df8_1.groupby(['lon','lat']).sum()
df8_2 = df8_2.reset_index()
df8_2 = df8_2.reindex(columns=['lat','lon','mean'])

# 유동인구를 격자에 할당
list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

df8_1 = pd.DataFrame({'격자명':list_tmp})
df8_1['유동인구'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    if i % 100 == 0:
        print('Process: ',i,'단계')
    
    for j in range(len(df8_2)):
        if (df8_2.loc[j, 'lat'] >= min_lat) & (df8_2.loc[j, 'lat'] <= max_lat) & (df8_2.loc[j, 'lon'] >= min_lon) & (df8_2.loc[j, 'lon'] <= max_lon):
            df8_1.loc[i,'유동인구'] = df8_1.loc[i,'유동인구'] + df8_2.loc[j,'mean']
df8_1

,격자명,유동인구
0,다사551085,0.000000
1,다사551086,1.248571
2,다사551087,11.122857
3,다사551088,4.014286
4,다사552085,0.262857
...,...,...
4505,다사646052,0.000000
4506,다사646053,0.000000
4507,다사647050,0.000000
4508,다사647051,0.000000


### df9: 어린이보호구역개수

In [11]:
# 9. 오산시_어린이보호구역

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

df9_1 = pd.DataFrame({'격자명':list_tmp})
df9_1['어린이보호구역개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df9)):
        if (df9.loc[j, '보호구역_위도'] >= min_lat) & (df9.loc[j, '보호구역_위도'] <= max_lat) & (df9.loc[j, '보호구역_경도'] >= min_lon) & (df9.loc[j, '보호구역_경도'] <= max_lon):
            df9_1.loc[i,'어린이보호구역개수'] = df9_1.loc[i,'어린이보호구역개수']+1
df9_1

,격자명,어린이보호구역개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


### df9: 오산시 어린이보호구역CCTV개수

In [12]:
# 9. 오산시_어린이보호구역

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

df9_2 = pd.DataFrame({'격자명':list_tmp})
df9_2['어린이보호구역CCTV개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df9)):
        if (df9.loc[j, '보호구역_위도'] >= min_lat) & (df9.loc[j, '보호구역_위도'] <= max_lat) & (df9.loc[j, '보호구역_경도'] >= min_lon) & (df9.loc[j, '보호구역_경도'] <= max_lon):
            df9_2.loc[i,'어린이보호구역CCTV개수'] = df9_2.loc[i,'어린이보호구역CCTV개수']+df9.loc[j, 'CCTV설치대수']
df9_2

,격자명,어린이보호구역CCTV개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


### df10: 초등학교개수

In [13]:
# 10. 오산시_학교위치정보

df10 = df10[df10['학교구분'] == '초등학교'].reset_index() # 초등학교 개수만 필터링

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df10_1 = pd.DataFrame({'격자명':list_tmp})
df10_1['초등학교개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df10)):
        if (df10.loc[j, '학교위치_위도'] >= min_lat) & (df10.loc[j, '학교위치_위도'] <= max_lat) & (df10.loc[j, '학교위치_경도'] >= min_lon) & (df10.loc[j, '학교위치_경도'] <= max_lon):
            df10_1.loc[i,'초등학교개수'] = df10_1.loc[i,'초등학교개수']+1
df10_1

,격자명,초등학교개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


### df13: 유치원/어린이집개수

In [14]:
# 13. 오산시_어린이집_유치원현황

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df13_1 = pd.DataFrame({'격자명':list_tmp})
df13_1['유치원/어린이집개수'] = 0


for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df13)):
        if (df13.loc[j, '시설위치_위도'] >= min_lat) & (df13.loc[j, '시설위치_위도'] <= max_lat) & (df13.loc[j, '시설위치_경도'] >= min_lon) & (df13.loc[j, '시설위치_경도'] <= max_lon):
            df13_1.loc[i,'유치원/어린이집개수'] = df13_1.loc[i,'유치원/어린이집개수']+1
df13_1

,격자명,유치원/어린이집개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


### df15: 무인교통단속카메라개수

In [15]:
# 15. 오산시_무인교통단속카메라

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

df15_1 = pd.DataFrame({'격자명':list_tmp})
df15_1['무인교통단속카메라개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df15)):
        if (df15.loc[j, '설치위치_위도'] >= min_lat) & (df15.loc[j, '설치위치_위도'] <= max_lat) & (df15.loc[j, '설치위치_경도'] >= min_lon) & (df15.loc[j, '설치위치_경도'] <= max_lon):
            df15_1.loc[i,'무인교통단속카메라개수'] = df15_1.loc[i,'무인교통단속카메라개수']+1
df15_1

,격자명,무인교통단속카메라개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


### df16: 도로안전표지판개수

In [16]:
# 16. 오산시_도로안전표지표준데이터

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

df16_1 = pd.DataFrame({'격자명':list_tmp})
df16_1['도로안전표지판개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df16)):
        if (df16.loc[j, 'lat'] >= min_lat) & (df16.loc[j, 'lat'] <= max_lat) & (df16.loc[j, 'lon'] >= min_lon) & (df16.loc[j, 'lon'] <= max_lon):
            df16_1.loc[i,'도로안전표지판개수'] = df16_1.loc[i,'도로안전표지판개수']+1
df16_1

,격자명,도로안전표지판개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


### df17: 횡단보도개수

In [17]:
# 17. 오산시_횡단보도

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df17_1 = pd.DataFrame({'격자명':list_tmp})
df17_1['횡단보도개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df17['features'])):
        for k in range(len(df17['features'][j]['geometry']['coordinates'][0][0])):
            lat = df17['features'][j]['geometry']['coordinates'][0][0][k][1]
            lon = df17['features'][j]['geometry']['coordinates'][0][0][k][0]
        
        
            if (lat >= min_lat) & (lat <= max_lat) & (lon >= min_lon) & (lon <= max_lon):
                df17_1.loc[i,'횡단보도개수'] = df17_1.loc[i,'횡단보도개수']+1
                break
df17_1

,격자명,횡단보도개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


### df18: 과속방지턱개수

In [18]:
# 18. 오산시_과속방지턱표준데이터

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df18_1 = pd.DataFrame({'격자명':list_tmp})
df18_1['과속방지턱개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df18)):
        if (df18.loc[j, '설치위치_위도'] >= min_lat) & (df18.loc[j, '설치위치_위도'] <= max_lat) & (df18.loc[j, '설치위치_경도'] >= min_lon) & (df18.loc[j, '설치위치_경도'] <= max_lon):
            df18_1.loc[i,'과속방지턱개수'] = df18_1.loc[i,'과속방지턱개수']+1
df18_1

,격자명,과속방지턱개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


### df19: 신호등개수

In [19]:
# 19. 오산시_신호등

# geojson내 coordinate를 dataframe화
list_tmp = []

for i in range(len(df19['features'])):
    tmp = df19['features'][i]['geometry']['coordinates']
    list_tmp.extend(tmp)
    
df19_tmp = pd.DataFrame(columns=['설치위치_위도','설치위치_경도'])

for i in range(0, 2*len(df19['features']), 2):
    a = list_tmp[i]
    b = list_tmp[i+1]
    tmp = {'설치위치_위도':b, '설치위치_경도':a}
    df19_tmp = df19_tmp.append(tmp, ignore_index=True)

# 격자에 할당
list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df19_1 = pd.DataFrame({'격자명':list_tmp})
df19_1['신호등개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df19_tmp)):
        if (df19_tmp.loc[j, '설치위치_위도'] >= min_lat) & (df19_tmp.loc[j, '설치위치_위도'] <= max_lat) & (df19_tmp.loc[j, '설치위치_경도'] >= min_lon) & (df19_tmp.loc[j, '설치위치_경도'] <= max_lon):
            df19_1.loc[i,'신호등개수'] = df19_1.loc[i,'신호등개수']+1
df19_1

,격자명,신호등개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


### df22: 버스정류장개수

In [20]:
# 22. 오산시_버스정류장

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df22_1 = pd.DataFrame({'격자명':list_tmp})
df22_1['버스정류장개수'] = 0


for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df22)):
        if (df22.loc[j, '정류장 위치_위도'] >= min_lat) & (df22.loc[j, '정류장 위치_위도'] <= max_lat) & (df22.loc[j, '정류장 위치_경도'] >= min_lon) & (df22.loc[j, '정류장 위치_경도'] <= max_lon):
            df22_1.loc[i,'버스정류장개수'] = df22_1.loc[i,'버스정류장개수']+1
df22_1

,격자명,버스정류장개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


### df27: 건물개수

In [21]:
# 27. 오산시_도로명주소_건물

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df27_1 = pd.DataFrame({'격자명':list_tmp})
df27_1['건물개수'] = 0


for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0] 
    
    for j in range(len(df27['features'])):
        for k in range(len(df27['features'][j]['geometry']['coordinates'][0][0])):
            lat = df27['features'][j]['geometry']['coordinates'][0][0][k][1]
            lon = df27['features'][j]['geometry']['coordinates'][0][0][k][0]
        
        
            if (lat >= min_lat) & (lat <= max_lat) & (lon >= min_lon) & (lon <= max_lon):
                df27_1.loc[i,'건물개수'] = df27_1.loc[i,'건물개수']+1
                break
df27_1

,격자명,건물개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


### df29: 체육시설개수

In [22]:
# 29. 오산시_체육시설현황

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df29_1 = pd.DataFrame({'격자명':list_tmp})
df29_1['체육시설개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df29)):
        if (df29.loc[j, '설치위치_위도'] >= min_lat) & (df29.loc[j, '설치위치_위도'] <= max_lat) & (df29.loc[j, '설치위치_경도'] >= min_lon) & (df29.loc[j, '설치위치_경도'] <= max_lon):
            df29_1.loc[i,'체육시설개수'] = df29_1.loc[i,'체육시설개수']+1
df29_1

,격자명,체육시설개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


### df30: 학원교습소개수

In [23]:
# 30. 오산시_학원_및_교습소_현황

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df30_1 = pd.DataFrame({'격자명':list_tmp})
df30_1['학원교습소개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df30)):
        if (df30.loc[j, '시설위치_위도'] >= min_lat) & (df30.loc[j, '시설위치_위도'] <= max_lat) & (df30.loc[j, '시설위치_경도'] >= min_lon) & (df30.loc[j, '시설위치_경도'] <= max_lon):
            df30_1.loc[i,'학원교습소개수'] = df30_1.loc[i,'학원교습소개수']+1
df30_1

,격자명,학원교습소개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


### 3. MultiLineString형 데이터 전처리

### MultiLineString을 Buffer를 활용하여 Polygon형태로 변환

In [24]:
# 100m x 100m 격자 불러오기
df_100m = gpd.read_file('4.오산시_연령별_거주인구격자(총인구).geojson')
df_100m = df_100m.drop('val',axis=1) # 여기서 경도, 위도, geometry만 가져온다
 
# geopandas를 활용하여 상세도로망 데이터를 geojson --> csv
df23_1 = gpd.read_file('23.오산시_상세도로망_LV6.geojson')

# multilinestring --> buffer 변환
for i in range(len(df23_1)):
    df23_1.loc[i,'geometry'] = df23_1.loc[i,'geometry'].buffer(0.00005)

df23_1

,link_id,max_speed,road_name,road_no,road_rank,link_type,road_type,facil_name,up_lanes,dw_lanes,oneway,length,width,car_lane,num_cross,barrier,geometry
0,478344684,60,수도권제2순환고속도로(봉담동탄,400,108,2,0,None,1,0,1,0.472,1,0,0,0,"POLYGON ((126.99785 37.17408, 126.99793 37.174..."
1,478344686,60,수도권제2순환고속도로(봉담동탄,400,108,2,0,None,2,0,1,0.606,2,0,0,0,"POLYGON ((126.99775 37.17394, 126.99775 37.173..."
2,478344690,60,수도권제2순환고속도로(봉담동탄,400,108,2,0,None,1,0,1,0.383,1,0,0,0,"POLYGON ((127.00101 37.17995, 127.00046 37.179..."
3,478344696,60,오산화성고속도로,171,108,2,0,None,1,0,1,0.488,1,0,0,0,"POLYGON ((126.99635 37.17614, 126.99635 37.176..."
4,478344700,40,오산화성고속도로,171,108,2,0,None,1,0,1,0.28,1,0,0,0,"POLYGON ((126.99753 37.17710, 126.99753 37.177..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,571376981,100,수도권제2순환고속도로(봉담동탄,400,101,1,0,None,2,0,1,0.643,2,1,0,0,"POLYGON ((127.04672 37.18057, 127.04673 37.180..."
1871,571376982,100,수도권제2순환고속도로(봉담동탄,400,101,1,0,None,2,0,1,0.936,2,1,0,0,"POLYGON ((127.04781 37.18097, 127.04781 37.180..."
1872,571376983,100,수도권제2순환고속도로(봉담동탄,400,101,1,2,세교지하차도,2,0,1,0.093,2,1,0,0,"POLYGON ((127.04768 37.18092, 127.04769 37.180..."
1873,900000166,0,None,0,107,32768,0,None,1,0,1,0.065,1,0,0,0,"POLYGON ((127.06014 37.16956, 127.06036 37.169..."


### df24: 전체 추정교통량

In [25]:
# 시간적 범위는 어린이가 가장 많이 몰리는 시간대인 14~20시로 한정
df24_1 = df24[df24['시간적범위'].isin(['15','16','17','18','19','20'])] 

# 상세도로망별 평균 "전체 추정교통량"도출
df24_2 = df24_1.groupby('상세도로망_LinkID').mean()[['전체 추정교통량']]
df24_2 = df24_2.reset_index()

# df23: 상세도로망 데이터와 연관짓기 위해 뒤에 상세도로망_LinkID2자리 삭제
for i in range(len(df24_2)):
    df24_2.loc[i,'상세도로망_LinkID'] = str(df24_2.iloc[i,0])[:-2]

# 뒤에 2자리를 지운 후 다시 groupby 실시
df24_3 = df24_2.groupby('상세도로망_LinkID').mean()[['전체 추정교통량']]
df24_3 = df24_3.reset_index()

# df23과 df24의 link_id의 열이름 통일
df24_3 = df24_3.rename(columns={'상세도로망_LinkID':'link_id'})

# link_id를 활용해 두 Dataframe을 merge
df_traffic = pd.merge(df23_1[['link_id','geometry']], df24_3, on='link_id', how='outer')
df_traffic = df_traffic.dropna()

# 격자별 "전체 추정교통량"을 0으로 초기화
df_100m['전체 추정교통량'] = 0

# 인덱스 초기화
df_traffic = df_traffic.reset_index(drop=True)

# 격자에 할당
for k in range(len(df_traffic)):
    buffer = df_traffic['geometry'][k].buffer(0.0001)
        
    for i in list(df_100m[df_100m['geometry'].intersection(buffer).is_empty == False]['전체 추정교통량'].index):
        df_100m.loc[i,'전체 추정교통량'] += df_traffic.loc[k,'전체 추정교통량']

### df25: 혼잡빈도강도

In [26]:
# 격자별 혼잡빈도강도 초기화
df_100m['혼잡빈도강도'] = 0

# 필요한 column만 불러온 후 groupby
df25_1 = df25[['상세도로망_LinkID','혼잡빈도강도']]

df25_2 = df25_1.groupby('상세도로망_LinkID').mean()[['혼잡빈도강도']]
df25_2 = df25_2.reset_index()

# link_ID 끝 두자리 제거
for i in range(len(df25_2)):
    df25_2.loc[i,'상세도로망_LinkID'] = str(df25_2.iloc[i,0])[:-2]
    
# groupby 및 merge
df25_3 = df25_2.groupby('상세도로망_LinkID').mean()[['혼잡빈도강도']]
df25_3 = df25_3.reset_index()
df25_3 = df25_3.rename(columns={'상세도로망_LinkID':'link_id'})

df_traffic = pd.merge(df23_1[['link_id','geometry']], df25_3, on='link_id', how='outer')
df_traffic = df_traffic.dropna()
df_traffic = df_traffic.reset_index(drop=True)

# 격자에 할당
for k in range(len(df_traffic)):
    buffer = df_traffic['geometry'][k].buffer(0.0001)
        
    for i in list(df_100m[df_100m['geometry'].intersection(buffer).is_empty == False]['혼잡빈도강도'].index):
        df_100m.loc[i,'혼잡빈도강도'] += df_traffic.loc[k,'혼잡빈도강도']

### df26: 혼잡시간강도

In [27]:
# 격자별 혼잡빈도강도 초기화 및 필요한 Column만 구분
df_100m['혼잡시간강도'] = 0
df26_1 = df26[['상세도로망_LinkID','혼잡시간강도']]

# 필요데이터 groupby 및 merge
df26_2 = df26_1.groupby('상세도로망_LinkID').mean()[['혼잡시간강도']]
df26_2 = df26_2.reset_index()

for i in range(len(df26_2)):
    df26_2.loc[i,'상세도로망_LinkID'] = str(df26_2.iloc[i,0])[:-2]

df26_3 = df26_2.groupby('상세도로망_LinkID').mean()[['혼잡시간강도']]
df26_3 = df26_3.reset_index()
df26_3 = df26_3.rename(columns={'상세도로망_LinkID':'link_id'})

df_traffic = pd.merge(df23_1[['link_id','geometry']], df26_3, on='link_id', how='outer')
df_traffic = df_traffic.dropna()
df_traffic = df_traffic.reset_index(drop=True)

# 격자에 할당
for k in range(len(df_traffic)):
    buffer = df_traffic['geometry'][k].buffer(0.0001)
        
    for i in list(df_100m[df_100m['geometry'].intersection(buffer).is_empty == False]['혼잡시간강도'].index):
        df_100m.loc[i,'혼잡시간강도'] += df_traffic.loc[k,'혼잡시간강도']
        
df_100m

,gid,geometry,전체 추정교통량,혼잡빈도강도,혼잡시간강도
0,다사551085,MULTIPOLYGON (((126.9942156468143 37.174182357...,0.0,0.0,0.0
1,다사551086,MULTIPOLYGON (((126.9942096381632 37.175083738...,0.0,0.0,0.0
2,다사551087,MULTIPOLYGON (((126.9942036292448 37.175985119...,0.0,0.0,0.0
3,다사551088,MULTIPOLYGON (((126.9941976200589 37.176886500...,0.0,0.0,0.0
4,다사552085,MULTIPOLYGON (((126.9953420754172 37.174187160...,0.0,0.0,0.0
...,...,...,...,...,...
4505,다사646052,MULTIPOLYGON (((127.1013836908642 37.144844715...,0.0,0.0,0.0
4506,다사646053,MULTIPOLYGON (((127.1013789603587 37.145746114...,0.0,0.0,0.0
4507,다사647050,MULTIPOLYGON (((127.1025191345573 37.143045698...,0.0,0.0,0.0
4508,다사647051,MULTIPOLYGON (((127.1025144178335 37.143947097...,0.0,0.0,0.0


## Concat

In [28]:
# 최종 데이터프레임 병합

df_final = pd.concat([df1_1.iloc[:,[0,1]],
                    df2_1.iloc[:,1],
                    df3_1.iloc[:,1],
                    df5_1.iloc[:,1],
                    df6_1.iloc[:,1],
                    df7_1.iloc[:,1],
                    df8_1.iloc[:,1],
                    df9_1.iloc[:,1],
                    df9_2.iloc[:,1],
                    df10_1.iloc[:,1],
                    df13_1.iloc[:,1],
                    df15_1.iloc[:,1],
                    df16_1.iloc[:,1],
                    df17_1.iloc[:,1],
                    df18_1.iloc[:,1],
                    df19_1.iloc[:,1],
                    df22_1.iloc[:,1],
                    df_100m.iloc[:,[2,3,4]],
                    df27_1.iloc[:,1],
                    df28_1.iloc[:,1],
                    df29_1.iloc[:,1],
                    df30_1.iloc[:,1]], axis=1)

df_final

,격자명,주정차단속건수,사고횟수,차량개수,유소년인구수,생산가능인구수,고령인구수,유동인구,어린이보호구역개수,어린이보호구역CCTV개수,...,과속방지턱개수,신호등개수,버스정류장개수,전체 추정교통량,혼잡빈도강도,혼잡시간강도,건물개수,건물연면적,체육시설개수,학원교습소개수
0,다사551085,0,0,0,0.0,0.0,0.0,0.000000,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0
1,다사551086,0,0,0,0.0,0.0,0.0,1.248571,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0
2,다사551087,0,0,0,0.0,0.0,0.0,11.122857,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0
3,다사551088,0,0,0,0.0,0.0,0.0,4.014286,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0
4,다사552085,0,0,0,0.0,0.0,0.0,0.262857,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4505,다사646052,0,0,0,0.0,0.0,0.0,0.000000,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0
4506,다사646053,0,0,0,0.0,0.0,0.0,0.000000,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0
4507,다사647050,0,0,0,0.0,0.0,0.0,0.000000,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0
4508,다사647051,0,0,0,0.0,0.0,0.0,0.000000,0,0,...,0,0,0,0.0,0.0,0.0,0,0.0,0,0


# 모델링 절차

## 1. 세부 데이터 설정

어린이보호구역 외의 어린이 교통사고 위험지역 20개를 선정하기 위해, 이미 어린이보호 구역이 포함된 격자는 제외하고 다음의 후보군 격자와 모델링 격자를 구축하였다.

1. 어린이 교통사고 위험지역 후보군을 설정한다.  

    (다음의 조건 중 1개 이상 만족 시, 후보군)
    - 격자 내 초등학교가 1개 이상 
    - 격자 내 유치원 및 어린이집이 1개 이상
    - 격자 내 학원 및 교습소가 1개 이상


2. 모델을 통해 훈련시킬 격자 데이터는 기존 어린이보호구역과 후보군을 제외한다. 

## 2. Feature selection

1. Logistic Regression 모델을 통한 P-VALUE로 유의미하지 않은 변수제거 (a = 0.1)
 - p-value 0.1 이상의 변수를 제거한다.
2. 다중공선성을 띄는 변수를 분산 인플레이션 계수(VIF)를 통해 제거 
 - VIF Factor 7이상의 변수를 제거한다.

## 3. Modeling

3가지 모델을 수립 후, 성능이 우수한 모델로 설정한 후보군의 사고발생확률 도출
1. KNN Classfier
2. RandomForest Classifier
3. LightGbm

## 4. 후보군 격자들의 사고 발생 확률 예측

가장 성능이 우수한 모델을 통해 어린이 교통사고 위험지역 20개소를 예측하고, 확률 순으로 내림차순하여 시각화한다.

## 5. 어린이 교통사고 위험지역 20개소 선정

    - 상위 TOP 20 선정 순서

    1. 기존 어린이보호 구역의 시설명이 포함된 격자의 경우, 후보에서 제거한다.
    2. 다음의 사고 확률이 높은 새로운 후보를 등록한 후, 다시 기존 어린이보호 구역의 시설명이 포함되는지 비교한다. 
    3. 1과 2의 작업을 기존 어린이보호 구역의 시설명이 포함되지 않을 때까지 반복한다.


In [29]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score
import lightgbm as lgb
import time
import random

### 1. 세부 데이터 설정

#### 유치원 및 어린이집 변수 추가하여 후보군 생성 - df_child

    (다음의 조건 중 1개 이상 만족 시, 후보군)
    - 격자 내 초등학교가 1개 이상 
    - 격자 내 유치원 및 어린이집이 1개 이상
    - 격자 내 학원 및 교습소가 1개 이상
앞서 할당시킨 4510개의 격자 데이터에, **어린이 대상**의 오산시_학원_및_교습소_현황 데이터를 추가한다.

In [30]:
df30['교습과정명'].unique()

array(['음악', '미술', '보습', ..., '영어 고급', '국수사과', '피아노(고급주3회)'], dtype=object)

많은 학원 유형 중, 어린이 교통사고에 한정시키기 위해 **'초등', '저학년', '초1'...'초6'이 들어간 것**으로 선정<br>
이들 키워드 중 1개라도 속하면 초등학생 학원이라고 분류하였다.

In [31]:
cond1 = df30['교습과정명'].str.contains('초등', na=False)
cond2 = df30['교습과정명'].str.contains('저학년', na=False)
cond3 = df30['교습과정명'].str.contains('초1', na=False)
cond4 = df30['교습과정명'].str.contains('초2', na=False)
cond5 = df30['교습과정명'].str.contains('초3', na=False)
cond6 = df30['교습과정명'].str.contains('초4', na=False)
cond7 = df30['교습과정명'].str.contains('초5', na=False)
cond8 = df30['교습과정명'].str.contains('초6', na=False)

df30_1 = df30.loc[cond1 | cond2 | cond3 | cond4 | cond5 | cond6 | cond7 | cond8,:]
df30_1 = df30_1.reset_index(drop=True)
df30_1

,업종구분명,교습과정명,시설위치_경도,시설위치_위도
0,학교교과교습학원,종합(초등),127.046027,37.147560
1,학교교과교습학원,수학+논술저학년,127.068752,37.133925
2,학교교과교습학원,수학초등4,127.068528,37.135115
3,학교교과교습학원,영어초등1,127.068528,37.135115
4,학교교과교습학원,수학초등1,127.068528,37.135115
...,...,...,...,...
295,학교교과교습학원,수학 초등,127.086780,37.142726
296,학교교과교습학원,초등수학A,127.061831,37.162835
297,학교교과교습학원,초등수학B,127.061831,37.162835
298,학교교과교습학원,영어초등,127.044224,37.143893


유치원생 및 초등학생 학원의 수를 격자화한다.

In [32]:
# 유치원생 및 초등학생 학원의 수를 격자화1
list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df30_2 = pd.DataFrame({'격자명':list_tmp})
df30_2['유치원및초등생학원개수'] = 0
df30_2

,격자명,유치원및초등생학원개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


In [33]:
# 유치원생 및 초등학생 학원의 수를 격자화2

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df30_1)):
        if (df30_1.loc[j, '시설위치_위도'] >= min_lat) & (df30_1.loc[j, '시설위치_위도'] <= max_lat) & (df30_1.loc[j, '시설위치_경도'] >= min_lon) & (df30_1.loc[j, '시설위치_경도'] <= max_lon):
            df30_2.loc[i,'유치원및초등생학원개수'] = df30_2.loc[i,'유치원및초등생학원개수']+1
df30_2

,격자명,유치원및초등생학원개수
0,다사551085,0
1,다사551086,0
2,다사551087,0
3,다사551088,0
4,다사552085,0
...,...,...
4505,다사646052,0
4506,다사646053,0
4507,다사647050,0
4508,다사647051,0


기존 df_final 데이터와 학원/교습소 정보를 추가한 데이터프레임을 병합한다.

In [34]:
df_final2 = pd.concat([df_final,df30_2.iloc[:,1]], axis=1)
df_final2

,격자명,주정차단속건수,사고횟수,차량개수,유소년인구수,생산가능인구수,고령인구수,유동인구,어린이보호구역개수,어린이보호구역CCTV개수,...,신호등개수,버스정류장개수,전체 추정교통량,혼잡빈도강도,혼잡시간강도,건물개수,건물연면적,체육시설개수,학원교습소개수,유치원및초등생학원개수
0,다사551085,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
1,다사551086,0,0,0,0,0,0,1.248571,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
2,다사551087,0,0,0,0,0,0,11.122857,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
3,다사551088,0,0,0,0,0,0,4.014286,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
4,다사552085,0,0,0,0,0,0,0.262857,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4505,다사646052,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
4506,다사646053,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
4507,다사647050,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
4508,다사647051,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0


### 기존 어린이보호구역 제거

In [35]:
# 기존 어린이 보호구역이 1개라도 있는 격자 제거
data = df_final2[df_final2['어린이보호구역개수'] >=1]
data  # 88개 격자 제거

,격자명,주정차단속건수,사고횟수,차량개수,유소년인구수,생산가능인구수,고령인구수,유동인구,어린이보호구역개수,어린이보호구역CCTV개수,...,신호등개수,버스정류장개수,전체 추정교통량,혼잡빈도강도,혼잡시간강도,건물개수,건물연면적,체육시설개수,학원교습소개수,유치원및초등생학원개수
391,다사571087,0,0,9,0,112,0,10.381429,1,0,...,0,0,0.000000,0.000,0.000,1,4540.02,0,0,0
703,다사579109,0,0,7,0,8,0,82.764286,1,1,...,0,0,6049.287167,145.575,221.190,10,313.94,0,0,0
721,다사580079,0,0,0,0,0,0,32.241429,1,3,...,1,0,610.706000,222.775,299.975,2,9631.56,0,0,0
998,다사584110,2,0,0,8,49,0,263.704286,1,2,...,0,0,1032.541667,92.465,171.620,2,6228.81,0,0,0
1129,다사586100,0,0,15,0,25,6,30.795714,1,1,...,0,0,0.000000,0.000,0.000,15,289.52,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4052,다사630043,0,0,14,0,11,9,308.422857,1,1,...,0,1,225.266000,30.010,50.955,39,257.09,0,0,0
4087,다사631039,1,0,0,0,0,0,300.541429,1,2,...,0,1,734.996667,99.575,99.885,5,0.00,0,0,0
4129,다사632041,0,0,0,0,0,0,131.621429,1,1,...,0,0,10976.046670,119.770,136.790,1,784.03,0,0,0
4281,다사636049,0,0,0,0,0,0,24.171429,1,2,...,0,0,0.000000,0.000,0.000,1,4669.25,0,0,0


In [36]:
# 위 index를 제외한 index 
data_idx = df_final2.index.drop(list(data.index))

# 위에서 추출한 index를 df에 적용
df_final2 = df_final2.loc[data_idx,:]
df_final2 # 기존 어린이보호구역이 제거된 df_final2

,격자명,주정차단속건수,사고횟수,차량개수,유소년인구수,생산가능인구수,고령인구수,유동인구,어린이보호구역개수,어린이보호구역CCTV개수,...,신호등개수,버스정류장개수,전체 추정교통량,혼잡빈도강도,혼잡시간강도,건물개수,건물연면적,체육시설개수,학원교습소개수,유치원및초등생학원개수
0,다사551085,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
1,다사551086,0,0,0,0,0,0,1.248571,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
2,다사551087,0,0,0,0,0,0,11.122857,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
3,다사551088,0,0,0,0,0,0,4.014286,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
4,다사552085,0,0,0,0,0,0,0.262857,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4505,다사646052,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
4506,다사646053,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
4507,다사647050,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
4508,다사647051,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0


### 후보군 도출 - df_child

In [37]:
# 처음에 말했던 것 처럼 아래 원리에 따라 후보군을 지정해준다.
 
'''
- 후보군 구분
  (1) 초등학교, 유치원/어린이집, 학원이 1개라도 있는 격자만이 가능.
  (2) 추가적으로 어린이 보호구역이 1개라도 있는 격자는 후보 불가 (앞에서 제거함)
'''
# 초등학교, 유치원/어린이집, 학원이 1개라도 있는 격자만이 가능.
cond1 = df_final2['초등학교개수'] >= 1
cond2 = df_final2['유치원/어린이집개수'] >= 1
cond3 = df_final2['유치원및초등생학원개수'] >= 1
df_child = df_final2[cond1 | cond2 | cond3]
df_child

,격자명,주정차단속건수,사고횟수,차량개수,유소년인구수,생산가능인구수,고령인구수,유동인구,어린이보호구역개수,어린이보호구역CCTV개수,...,신호등개수,버스정류장개수,전체 추정교통량,혼잡빈도강도,혼잡시간강도,건물개수,건물연면적,체육시설개수,학원교습소개수,유치원및초등생학원개수
547,다사575106,0,0,0,0,0,0,36.305714,0,0,...,0,0,0.000000,0.000,0.000,5,3612.96,0,0,0
623,다사577109,0,0,42,0,45,6,215.321429,0,0,...,0,2,391.707167,98.790,141.115,26,754.94,0,15,8
630,다사578077,0,0,0,96,336,26,291.098571,0,0,...,0,0,0.000000,0.000,0.000,3,8341.76,0,0,0
671,다사579077,0,0,0,197,558,35,137.675714,0,0,...,0,0,0.000000,0.000,0.000,3,9919.33,0,0,0
702,다사579108,0,0,26,0,50,9,57.368571,0,0,...,0,0,0.000000,0.000,0.000,17,303.44,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4290,다사636058,77,0,15,92,169,15,50.320000,0,0,...,3,0,729.415000,165.395,235.695,2,7913.50,1,14,1
4348,다사638056,0,0,1208,91,304,26,16.457143,0,0,...,0,0,0.000000,0.000,0.000,2,7903.55,0,0,0
4349,다사638057,0,0,0,56,204,15,35.874286,0,0,...,0,0,0.000000,0.000,0.000,3,9847.49,0,0,0
4350,다사638058,0,0,1450,152,596,40,48.058571,0,0,...,0,0,0.000000,0.000,0.000,5,9655.33,0,0,0


In [38]:
df_child[['초등학교개수','유치원/어린이집개수','유치원및초등생학원개수']][cond1 | cond2 | cond3] # 조건만족하는지 확인

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,초등학교개수,유치원/어린이집개수,유치원및초등생학원개수
547,0,1,0
623,0,0,8
630,0,1,0
671,0,2,0
702,0,1,0
...,...,...,...
4290,0,0,1
4348,0,2,0
4349,0,1,0
4350,0,1,0


In [39]:
# 위 253개의 index를 제외한 나머지가 바로 train의 index1 --> train의 index만 따로 추출
data2_idx = df_final2.index.drop(list(df_child.index))

# 위에서 추출한 index를 df에 적용
data_real = df_final2.loc[data2_idx,:]
data_real

,격자명,주정차단속건수,사고횟수,차량개수,유소년인구수,생산가능인구수,고령인구수,유동인구,어린이보호구역개수,어린이보호구역CCTV개수,...,신호등개수,버스정류장개수,전체 추정교통량,혼잡빈도강도,혼잡시간강도,건물개수,건물연면적,체육시설개수,학원교습소개수,유치원및초등생학원개수
0,다사551085,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
1,다사551086,0,0,0,0,0,0,1.248571,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
2,다사551087,0,0,0,0,0,0,11.122857,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
3,다사551088,0,0,0,0,0,0,4.014286,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
4,다사552085,0,0,0,0,0,0,0.262857,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4505,다사646052,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
4506,다사646053,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
4507,다사647050,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0
4508,다사647051,0,0,0,0,0,0,0.000000,0,0,...,0,0,0.0,0.0,0.0,0,0.0,0,0,0


In [40]:
data_real.columns

Index(['격자명', '주정차단속건수', '사고횟수', '차량개수', '유소년인구수', '생산가능인구수', '고령인구수', '유동인구',
       '어린이보호구역개수', '어린이보호구역CCTV개수', '초등학교개수', '유치원/어린이집개수', '무인교통단속카메라개수',
       '도로안전표지판개수', '횡단보도개수', '과속방지턱개수', '신호등개수', '버스정류장개수', '전체 추정교통량',
       '혼잡빈도강도', '혼잡시간강도', '건물개수', '건물연면적', '체육시설개수', '학원교습소개수',
       '유치원및초등생학원개수'],
      dtype='object')

모델링에 사용하지 않는 변수 삭제
- 유치원및 초등학생학원개수

### 253개 후보군 - df_child

In [41]:
# 후보군에서의 어린이 보호구역 개수,유치원및초등생학원개수 column 삭제

df_child.drop(['유치원및초등생학원개수'],axis=1,inplace=True)
df_child.shape

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(253, 25)

### 4169개 모델링 대상 - df_real

In [42]:
data_real.drop(['유치원및초등생학원개수'],axis=1,inplace=True)
data_real.shape

(4169, 25)

In [43]:
data_real.reset_index(drop=True,inplace=True)
data_real.columns

Index(['격자명', '주정차단속건수', '사고횟수', '차량개수', '유소년인구수', '생산가능인구수', '고령인구수', '유동인구',
       '어린이보호구역개수', '어린이보호구역CCTV개수', '초등학교개수', '유치원/어린이집개수', '무인교통단속카메라개수',
       '도로안전표지판개수', '횡단보도개수', '과속방지턱개수', '신호등개수', '버스정류장개수', '전체 추정교통량',
       '혼잡빈도강도', '혼잡시간강도', '건물개수', '건물연면적', '체육시설개수', '학원교습소개수'],
      dtype='object')

종속변수인 사고횟수를 격자 내에서 1번이라도 일어나면 1, 그 외는 0으로 할당한다.

In [44]:
for i in range(len(data_real)):
    if data_real.loc[i,'사고횟수'] >= 1:
        data_real.loc[i,'사고횟수'] = 1


#### 모델생성용: 4169개 격자, 위험지역 후보군: 253개 격자

In [45]:
# 추후 격자명을 분석하기 위해 복사본 생성
data1_original = data_real.copy()
df_child_original = df_child.copy()
# 격자명 변수 삭제
data_real.drop('격자명',axis=1,inplace=True)
df_child.drop('격자명',axis=1,inplace=True)


In [46]:
data_real.shape, df_child.shape

((4169, 24), (253, 24))

In [47]:
# 사고횟수를 y로, 그 외 변수는 모두 x

feature_columns = list(data_real.columns)
feature_columns.remove('사고횟수')

data_x = data_real[feature_columns]
data_y = data_real[['사고횟수']]
df_child_x = df_child[feature_columns]
df_child_y = df_child[['사고횟수']]
data_x.shape, data_y.shape, df_child_x.shape, df_child_y.shape

((4169, 23), (4169, 1), (253, 23), (253, 1))

#### 오버샘플링

종속변수가 편향되어 있으므로, **오버샘플링 기법**을 통해 종속변수의 개수 평준화를 한다.

우선, train set 과 test set을 나누고 모델링에 활용되는 train set에 대해서만 오버샘플링 기법을 적용한다.

In [48]:
# 종속변수 분포 확인
data_y.value_counts()

사고횟수
0       3900
1        269
dtype: int64

#### Train / test split

모델링을 평가하기 위해 4169개의 데이터를 train set, test set으로 나눈다.

In [49]:
# train vs test 나누기
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size = 0.2, random_state = 42, shuffle=True, stratify=data_y) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(3335, 23) (834, 23) (3335, 1) (834, 1)


In [50]:
train = pd.concat([train_x,train_y],axis=1)
train_0 = train[train['사고횟수'] == 0]
train_1 = train[train['사고횟수'] == 1]

train_1_over = train_1.sample(3120, replace=True, random_state=10)
train_over = pd.concat([train_0, train_1_over], axis=0)
train_over.사고횟수.value_counts()

1    3120
0    3120
Name: 사고횟수, dtype: int64

In [51]:
# 독립변수와 종속변수 분류
feature_columns = list(train_over.columns.difference(['사고횟수','격자명']))
X = train_over[feature_columns]
y = train_over[['사고횟수']]

# 이름을 기존과 동일하게 변경
train_x = X.copy()
train_y = y.copy()
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((6240, 23), (6240, 1), (834, 23), (834, 1))

## 2. Feature selection

### 변수 간의 상관계수를 도출하여 유의미하지 않은 변수 제거

1. 유치원/어린이집개수
2. 어린이보호구역개수
3. 초등학교개수
4. 어린이보호구역CCTV개수

In [52]:
train_x.corr()

,건물개수,건물연면적,고령인구수,과속방지턱개수,도로안전표지판개수,무인교통단속카메라개수,버스정류장개수,생산가능인구수,신호등개수,어린이보호구역CCTV개수,...,유치원/어린이집개수,전체 추정교통량,주정차단속건수,차량개수,체육시설개수,초등학교개수,학원교습소개수,혼잡빈도강도,혼잡시간강도,횡단보도개수
건물개수,1.000000,-0.045362,0.518960,0.091165,-0.006832,0.134808,0.105512,0.306288,0.005684,NaN,...,NaN,-0.084491,0.131646,0.161234,0.078125,NaN,0.128725,0.115673,0.100588,0.101763
건물연면적,-0.045362,1.000000,0.186949,0.000017,0.061097,0.014867,0.048193,0.296936,0.116574,NaN,...,NaN,-0.031431,0.044721,0.074151,0.075667,NaN,0.022302,0.031068,0.040514,0.136351
고령인구수,0.518960,0.186949,1.000000,0.087862,-0.063456,0.060432,0.097745,0.773324,0.082381,NaN,...,NaN,-0.109982,0.040456,0.271334,0.124715,NaN,0.106693,0.028310,0.028232,0.107422
과속방지턱개수,0.091165,0.000017,0.087862,1.000000,-0.031098,0.066192,-0.033553,0.087094,-0.033667,NaN,...,NaN,-0.032847,-0.001829,0.004985,0.010081,NaN,0.083934,0.111406,0.099572,0.026447
도로안전표지판개수,-0.006832,0.061097,-0.063456,-0.031098,1.000000,0.082782,0.141903,-0.069484,0.609353,NaN,...,NaN,0.406007,0.027641,-0.056833,-0.070382,NaN,-0.038001,0.491136,0.536307,0.475995
무인교통단속카메라개수,0.134808,0.014867,0.060432,0.066192,0.082782,1.000000,0.131032,0.076248,0.132825,NaN,...,NaN,-0.012021,0.165723,-0.002693,-0.014885,NaN,0.033417,0.152193,0.144447,0.197031
버스정류장개수,0.105512,0.048193,0.097745,-0.033553,0.141903,0.131032,1.000000,0.143909,0.264117,NaN,...,NaN,0.112729,0.028172,0.005775,-0.002412,NaN,0.053904,0.237526,0.227073,0.190159
생산가능인구수,0.306288,0.296936,0.773324,0.087094,-0.069484,0.076248,0.143909,1.000000,0.127002,NaN,...,NaN,-0.108903,0.175975,0.387249,0.076235,NaN,0.078723,0.068768,0.068756,0.196773
신호등개수,0.005684,0.116574,0.082381,-0.033667,0.609353,0.132825,0.264117,0.127002,1.000000,NaN,...,NaN,0.276013,0.085886,0.009060,-0.009398,NaN,-0.032628,0.471408,0.534213,0.713277
어린이보호구역CCTV개수,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
train_x = train_x.drop(['유치원/어린이집개수','어린이보호구역개수','초등학교개수','어린이보호구역CCTV개수'], axis=1)

### 설명변수 정규화

Logistic Regression을 진행하기 전 설명변수들의 각각의 변수값의 분포는 매우 다양하므로, 0과 1 사이의 값으로 정규화 진행한다.


In [54]:
# train data 정규화
min_max_scaler = preprocessing.MinMaxScaler()
train_x2 = min_max_scaler.fit_transform(train_x)
train_x2 = pd.DataFrame(train_x2, columns = train_x.columns)
train_y.reset_index(drop=True,inplace=True)

### Logistic Regression을 통한 P-VALUE로 유의미하지 않은 변수제거 (a = 0.1)

In [55]:
# 유의미하지 않은 변수를 선정하기위해 로지스틱 다중회귀분석을 통한 p-value를 활용 (기준값: a=0.1)
## 결론적으로 3개의 독립변수가 생존
train_x2 = add_constant(train_x2)
model = sm.Logit(train_y, train_x2)
fit_model = model.fit()
fit_model.summary(alpha=0.1)

Optimization terminated successfully.
         Current function value: 0.427618
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   사고횟수   No. Observations:                 6240
Model:                          Logit   Df Residuals:                     6220
Method:                           MLE   Df Model:                           19
Date:                Wed, 27 Jan 2021   Pseudo R-squ.:                  0.3831
Time:                        21:31:21   Log-Likelihood:                -2668.3
converged:                       True   LL-Null:                       -4325.2
Covariance Type:            nonrobust   LLR p-value:                     0.000
===============================================================================
                  coef    std err          z      P>|z|       [0.05       0.95]
-------------------------------------------------------------------------------
const          -1.9140      0.054    -35.216      0.000      -2.003      -1.825
건물개수            3.8480      0.360     10.676      0.000       3.255       4.441
건물연면적           5.8444      2.057      2.842      0.004       2.462       9.227
고령인구수           2.1248      0.718      2.961      0.003       0.944       3.305
과속방지턱개수         0.5621      0.375      1.498      0.134      -0.055       1.179
도로안전표지판개수       2.0797      0.318      6.532      0.000       1.556       2.603
무인교통단속카메라개수    -0.0587      0.380     -0.154      0.877      -0.684       0.567
버스정류장개수         2.1682      0.316      6.866      0.000       1.649       2.688
생산가능인구수         0.6720      0.980      0.686      0.493      -0.940       2.284
신호등개수           0.4610      0.471      0.980      0.327      -0.313       1.235
유동인구            2.9336      0.399      7.352      0.000       2.277       3.590
유소년인구수          0.7387      1.001      0.738      0.461      -0.908       2.385
전체 추정교통량        4.9592      0.438     11.316      0.000       4.238       5.680
주정차단속건수        -0.4619      0.959     -0.482      0.630      -2.039       1.115
차량개수           -0.1818      0.881     -0.206      0.837      -1.631       1.267
체육시설개수          2.8331      0.574      4.938      0.000       1.889       3.777
학원교습소개수        -0.2965      1.165     -0.254      0.799      -2.213       1.620
혼잡빈도강도         -1.1072      1.713     -0.646      0.518      -3.925       1.711
혼잡시간강도          5.1717      2.110      2.450      0.014       1.700       8.643
횡단보도개수          3.1546      0.368      8.568      0.000       2.549       3.760
===============================================================================
"""

### 분석 결과

Logistic Regression 결과, p-value가 0.1보다 작은 변수들은 해당 모델의 귀무가설(회귀계수 = 0) 을 기각하게 되므로 유의미한 변수이다.

따라서, p-value가 0.1보다 큰 변수들은 feature selection에서 제거한다.

제거된 변수는 다음과 같다.<br>
1. 과속방지턱개수 : p-value 0.134
2. 무인교통단속카메라개수 : p-value 0.877
3. 생산가능인구수 : p-value 0.493
4. 신호등개수 : p-value 0.327
5. 유소년인구수 : p-value 0.461
6. 주정차단속건수 : p-value 0.630
7. 차량개수 : p-value 0.837
8. 학원교습소개수 : p-value 0.799
9. 혼잡빈도강도 : p-value 0.518



### 독립변수 재설정

In [56]:
feature_columns = ['건물개수','건물연면적','고령인구수','버스정류장개수','도로안전표지판개수','유동인구','전체 추정교통량','체육시설개수','혼잡시간강도','횡단보도개수']
len(feature_columns)

10

In [57]:
train_x_final = train_x.loc[:,feature_columns]
train_y_final = train_y
test_x_final = test_x.loc[:,feature_columns]
test_y_final = test_y

df_child_x = df_child_x.loc[:,feature_columns] # 후보군 data도 똑같이 변환
print(train_x_final.shape, train_y_final.shape, test_x_final.shape, test_y_final.shape)

(6240, 10) (6240, 1) (834, 10) (834, 1)


###  분산 인플레이션 계수(VIF) 계산

VIF factor 가 7이상인 다중공선성을 띄는 변수를 제거한다.<br>


In [58]:
train_x_final = add_constant(train_x_final) 
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    train_x_final.values, i) for i in range(train_x_final.shape[1])]
vif["features"] = train_x_final.columns
vif

,VIF Factor,features
0,2.122527,const
1,1.519175,건물개수
2,1.136188,건물연면적
3,1.497319,고령인구수
4,1.132537,버스정류장개수
5,1.648744,도로안전표지판개수
6,1.509750,유동인구
7,1.569762,전체 추정교통량
8,1.032807,체육시설개수
9,2.374095,혼잡시간강도


In [59]:
# logistic regression 및 VIF에 들어갔던 상수항 제거
train_x_final = train_x_final.drop('const',axis=1)
train_x_final.shape, test_x_final.shape

((6240, 10), (834, 10))

#### 분석 결과

상관계수와 logistic regression 모델을 통해 제거되고 남은 변수들만의 다중공선성 효과가 미비하다고 할 수 있다.

따라서 **위의 10가지 변수에 대해 머신러닝 모델**을 수립하고자 한다.

모델링 과정은 다음과 같다.

## 3. Modeling

### 정규화

새로 정의한 10개의 변수에 대해 다시 정규화한다.

In [60]:
# train data & test data 정규화
min_max_scaler = preprocessing.MinMaxScaler()
train_x_final2 = min_max_scaler.fit_transform(train_x_final)
test_x_final2 = min_max_scaler.fit_transform(test_x_final)
df_child_x = min_max_scaler.fit_transform(df_child_x)

#정규화 결과는 array 이므로 DataFrame화
train_x_final2 = pd.DataFrame(train_x_final2, columns = train_x_final.columns)
test_x_final2 = pd.DataFrame(test_x_final2, columns = test_x_final.columns)
df_child_x = pd.DataFrame(df_child_x, columns = test_x_final.columns)

### KNN Classifier

In [61]:
# 성능 지표 (Accuracy, Precision, Recall, Specificity, F1-score)

def model_evaluation(label, predict):
    cf_matrix = confusion_matrix(label, predict)
    Accuracy = (cf_matrix[0][0] + cf_matrix[1][1]) / sum(sum(cf_matrix))
    Precision = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[0][1])
    Recall = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[1][0])
    Specificity = cf_matrix[0][0] / (cf_matrix[0][0] + cf_matrix[0][1])
    F1_Score = (2 * Recall * Precision) / (Recall + Precision)
    
    print("Accuracy: ", round(Accuracy,3))
    print("Precision: ", round(Precision,3))
    print("Recall: ", round(Recall,3))
    print("Specificity: ", round(Specificity,3))

In [62]:
# 혼동하지 않도록 변수명 통일
train_x = train_x_final2.copy()
train_y = train_y_final.copy()
test_x = test_x_final2.copy()
test_y = test_y_final.copy()

train_x = train_x.reset_index(drop=True)
train_y = train_y.reset_index(drop=True)
test_x = test_x.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

train_x.shape, train_y.shape, test_x.shape, test_y.shape

((6240, 10), (6240, 1), (834, 10), (834, 1))

In [63]:
# KNeighborsClassifier + 그리드서치로 최적 하이퍼 파라미터 선정(기준: Recall)

knn_parameters = [{
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
    'leaf_size': [1, 3, 5, 10, 15, 20, 25, 30, 35, 40],
    'weights': ['uniform', 'distance']}]
# Stratified k-fold (default for classifier)
# n = 5 folds is default
knn_classifier = KNeighborsClassifier()
grid_cv = GridSearchCV(estimator = knn_classifier, param_grid = knn_parameters, cv=10, scoring='recall')
grid_cv.fit(train_x, train_y)

pred = grid_cv.predict(test_x)
print('최적 하이퍼 파라미터: ', grid_cv.best_params_)

/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was

최적 하이퍼 파라미터:  {'leaf_size': 1, 'n_neighbors': 1, 'weights': 'uniform'}


/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [64]:
# 성능 확인
print(confusion_matrix(test_y, pred))
print(model_evaluation(test_y, pred))

[[722  58]
 [ 35  19]]
Accuracy:  0.888
Precision:  0.247
Recall:  0.352
Specificity:  0.926
None


### 모델 성능 평가

Accuracy는 0.888로 다소 높은 편이나, 본 과제의 목적은 사고 발생 위험지역을 선정하는 것이기 때문에, 다양한 성능 지표 중에서 **Recall** 에 집중하고자 한다.

**Recall 성능은 실제 사고 발생한 구역 중에서 모델이 사고 발생이라고 예측한 것의 비율을 나타내므로 본 주제와 가장 적합하다.**

KNN Classifier 모델의 Recall 성능은 0.352로 낮은 편에 속한다. 

### RFClassifier 모델링

In [65]:
# RandomForest Classifier & 그리드 서치로 최적 하이퍼 파라미터 선정 (기준: Recall)
# 최적 파라미터 값 찾기
params = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }
rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 5, n_jobs = -1, scoring='recall')
grid_cv.fit(train_x, train_y)

pred = grid_cv.predict(test_x)
print('최적 하이퍼 파라미터: ', grid_cv.best_params_)

/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/app-root/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was pa

최적 하이퍼 파라미터:  {'max_depth': 12, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}


In [66]:
# 성능 확인

print(confusion_matrix(test_y, pred))
print(model_evaluation(test_y, pred))

[[675 105]
 [ 20  34]]
Accuracy:  0.85
Precision:  0.245
Recall:  0.63
Specificity:  0.865
None


#### 모델 성능 평가

Recall 성능 측면에서는 0.63으로 Random Forest Classifier 모델은 KNN Classifier 모델보다 우수한 편에 속한다.


### lightgbm

In [67]:
# lightgbm 버전 상, 한글 feature를 읽지 못하여 X1 ~ X10 변수로 처리
train_x.columns = ['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10']
train_y.columns = ['S']

test_x.columns = ['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10']
test_y.columns = ['S']

In [68]:
# 모델링

start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 5, # 트리 깊이
             'num_leaves': 24,
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'metric': 'binary_logloss',
            'objective': 'binary', # 목적 함수
            'num_class': 1,
            'min_data_in_leaf': 400,
            'max_bin':1000} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
pred = np.round(lgb_model.predict(test_x),0) # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측
print("Accuracy: %.2f" % (accuracy_score(test_y, pred) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

[LightGBM] [Info] Number of positive: 3120, number of negative: 3120


/opt/app-root/lib/python3.6/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000670 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3491
[LightGBM] [Info] Number of data points in the train set: 6240, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

In [69]:
# 성능 확인

print(confusion_matrix(test_y, pred))
print(model_evaluation(test_y, pred))

[[583 197]
 [  8  46]]
Accuracy:  0.754
Precision:  0.189
Recall:  0.852
Specificity:  0.747
None


### 모델 성능 평가

Recall 성능 측면에서 0.852로 다른 두 모델보다 우월한 수치이다. <br>
다시 말하면, **54개의 실제 사고가 일어난 격자 중 46개의 격자를 사고가 일어날 것이라고 예측하여 맞춘 우수한 모델**이다.

따라서, 본 과제에서는 구축한 LightGBM 모델을 통해 253개의 후보군 격자들의 사고 발생 확률을 도출하였다.

## 4. 후보군 격자들의 사고 발생 확률 예측

In [70]:
# 격자의 중심좌표 불러오기 (경도 위도를 표현해주기 위해)
df2_1 = pd.DataFrame(columns=['격자명','경도','위도'])
for i in range(len(df2['features'])):
    a = df2['features'][i]['properties']['gid']
    c = np.mean(np.array(df2['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df2['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '경도': c,'위도': d}
    df2_1 = df2_1.append(new_data, ignore_index=True)

df2_1

,격자명,경도,위도
0,다사551085,126.994776,37.174635
1,다사551086,126.994770,37.175537
2,다사551087,126.994764,37.176438
3,다사551088,126.994758,37.177340
4,다사552085,126.995902,37.174640
...,...,...,...
4505,다사646052,127.101944,37.145297
4506,다사646053,127.101940,37.146199
4507,다사647050,127.103080,37.143498
4508,다사647051,127.103075,37.144400


In [71]:
df1 = df2_1.iloc[list(df_child.index),:] #기존 df_child index를 통해 격자명을 불러온다
df1.reset_index(drop=True, inplace=True)

#모델을 통해 예측
s1 = pd.Series(lgb_model.predict(df_child_x))
df1['prob'] = s1

#예측 확률을 내림차순으로 정렬
df_end = df1.sort_values(by='prob',ascending=False)
df_end


/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,격자명,경도,위도,prob
174,다사622056,127.074900,37.148809,0.798837
39,다사595087,127.044328,37.176639,0.798837
201,다사625064,127.078238,37.156032,0.798837
108,다사614079,127.065773,37.169508,0.798120
181,다사623056,127.076026,37.148813,0.798120
...,...,...,...,...
89,다사609069,127.060193,37.160474,0.254431
90,다사609109,127.059984,37.196529,0.254431
22,다사589060,127.037718,37.152276,0.199464
82,다사607073,127.057920,37.164071,0.183458


#### 사고 발생 지역 상위 20 도출

In [72]:
df_end.head(20)

,격자명,경도,위도,prob
174,다사622056,127.074900,37.148809,0.798837
39,다사595087,127.044328,37.176639,0.798837
201,다사625064,127.078238,37.156032,0.798837
108,다사614079,127.065773,37.169508,0.798120
181,다사623056,127.076026,37.148813,0.798120
179,다사623053,127.076041,37.146109,0.793109
42,다사597048,127.046791,37.141494,0.793109
173,다사622053,127.074915,37.146105,0.793109
196,다사624061,127.077127,37.153324,0.792377
41,다사596087,127.045455,37.176644,0.789931


#### 사고 발생 지역 하위 20 도출

In [73]:
df_end.tail(20)

,격자명,경도,위도,prob
75,다사605080,127.055630,37.170372,0.299785
60,다사601049,127.051290,37.142412,0.299785
249,다사638056,127.092917,37.148872,0.263555
245,다사636048,127.090704,37.141653,0.263555
52,다사599046,127.049054,37.139699,0.255944
91,다사610068,127.061325,37.159577,0.255944
251,다사638058,127.092907,37.150675,0.255944
250,다사638057,127.092912,37.149774,0.255944
4,다사579108,127.026187,37.195498,0.255944
5,다사580077,127.027487,37.167559,0.255944


In [74]:
# 283개의 평균 사고 발생 확률
df_end.prob.mean()

0.5887513078123618

## 5. 어린이 교통사고 위험지역 20개소 선정

시각화를 통해 기존 어린이보호 구역의 시설명과 동일한 격자의 경우, 후보에서 제거하고 21번째 후보를 20번째 후보로 할당시킨다.

위의 작업을 반복하여 기존 어린이보호구역 시설명과 겹치지 않는 조건을 만족하는 위험격자 20개를 도출한다.

In [75]:
#시각화를 위해 기존 격자의 MultiPolygon 데이터를 할당

df_MultiP = gpd.read_file('2.오산시_어린이교통사고_격자.geojson') # MutiPolygon 데이터를 가져오기위해 geopandas 를 이용
df_MultiP = df_MultiP[['gid','geometry']]
df_MultiP.rename(columns = {'gid':'격자명'}, inplace=True)

df_end = pd.merge(df_MultiP, df_end, on='격자명') #위험격자명과 MultiPolygon 데이터를 병합

df_end = df_end.sort_values(by='prob',ascending=False) # 사고 유발 확률로 내림차순 정렬

#후보군 교체를 위해 40개까지 할당
df_TOP40 = df_end.head(40)
df_TOP40.reset_index(drop=True,inplace=True) #인덱스별로 시각화하기 위해 초기화

In [76]:
# 시각화 분석
import folium

m = folium.Map(
    location=[37.1516633,127.0359803], # 지도의 정 중앙지점을 표시
    zoom_start=12 # 초기 줌의 정도 설정
)
# df31은 오산시 경계
style2 = {'fillColor': 'white'}
folium.GeoJson( 
    df31,  
    name='child_pro',
    style_function=lambda x:style2,
).add_to(m) #행정동 경계를 지도내에 add

def GeoJson_where(json):
    style = {'fillColor': 'Red'}
    folium.GeoJson( 
    json,
    tooltip=f'{i}번째 위험구역',
   style_function=lambda x:style,
    name='child_pro').add_to(m) #행정동 경계를 지도 내에 add
    
# 상위 20개 할당

for i in range(0,20): #0번 ~ 19번
    try:
        GeoJson_where(df_TOP40['geometry'][i])
    except:
        print(i)


for i in range(len(df9)): # 기존 어린이 보호구역을 지도내에 마커로 add
    folium.Marker([df9.iloc[i]['보호구역_위도'], df9.iloc[i]['보호구역_경도']],
                  tooltip =df9.iloc[i]['시설명'],
                  icon = folium.Icon(color='blue',icon='home')).add_to(m)

In [77]:
m

### 기존 어린이보호구역의 시설 위치와 동일한 격자 삭제

1번, 2번, 8번, 9번, 11번, 15번, 18번 위험지역 삭제 후, 상위 위험지역 7개 추가


In [78]:
# 7개 지역 삭제
df_TOP40 = df_TOP40.drop([1,2, 8, 9, 11, 15, 18],axis=0)

# 시각화
m = folium.Map(
    location=[37.1516633,127.0359803], # 지도의 정 중앙지점을 표시
    zoom_start=12 # 초기 줌의 정도 설정
)
# df31은 오산시 경계
style2 = {'fillColor': 'white'}
folium.GeoJson( 
    df31,  
    name='child_pro',
    style_function=lambda x:style2,
).add_to(m) #행정동 경계를 지도내에 add

for i in range(0,27): #0번 ~ 27번
    try:
        GeoJson_where(df_TOP40['geometry'][i])
    except:
        print(i,"번째 격자는 제거되었습니다")

for i in range(len(df9)): #어린이 보호구역을 지도내에 마커로 add
    folium.Marker([df9.iloc[i]['보호구역_위도'], df9.iloc[i]['보호구역_경도']],
              tooltip =df9.iloc[i]['시설명'],
              icon = folium.Icon(color='blue',icon='home')).add_to(m)

1 번째 격자는 제거되었습니다
2 번째 격자는 제거되었습니다
8 번째 격자는 제거되었습니다
9 번째 격자는 제거되었습니다
11 번째 격자는 제거되었습니다
15 번째 격자는 제거되었습니다
18 번째 격자는 제거되었습니다


In [79]:
m

## 재검토 후, 기존 어린이보호구역 시설 위치와 동일한 격자 삭제

26번 위험구역 삭제 후, 상위 1개 격자 추가

In [80]:
# 1개 지역 삭제
df_TOP40 = df_TOP40.drop([26],axis=0)

# 시각화
m = folium.Map(
    location=[37.1516633,127.0359803], # 지도의 정 중앙지점을 표시
    zoom_start=12 # 초기 줌의 정도 설정
)
# df31은 오산시 경계
style2 = {'fillColor': 'white'}
folium.GeoJson( 
    df31,  
    name='child_pro',
    style_function=lambda x:style2,
).add_to(m) #행정동 경계를 지도내에 add

for i in range(0,28): #0번 ~ 27번
    try:
        GeoJson_where(df_TOP40['geometry'][i])
    except:
        print(i,"번째 격자는 제거되었습니다")

for i in range(len(df9)): #어린이 보호구역을 지도내에 마커로 add
    folium.Marker([df9.iloc[i]['보호구역_위도'], df9.iloc[i]['보호구역_경도']],
              tooltip =df9.iloc[i]['시설명'],
              icon = folium.Icon(color='blue',icon='home')).add_to(m)

1 번째 격자는 제거되었습니다
2 번째 격자는 제거되었습니다
8 번째 격자는 제거되었습니다
9 번째 격자는 제거되었습니다
11 번째 격자는 제거되었습니다
15 번째 격자는 제거되었습니다
18 번째 격자는 제거되었습니다
26 번째 격자는 제거되었습니다


# 최종 결과



In [81]:
m.save('과제1_오산시위험지역20개.html')
m

In [82]:
df_end = df_TOP40[:20]
df_end

,격자명,geometry,경도,위도,prob
0,다사622056,"MULTIPOLYGON (((127.07434 37.14836, 127.07433 ...",127.074900,37.148809,0.798837
3,다사614079,"MULTIPOLYGON (((127.06521 37.16906, 127.06521 ...",127.065773,37.169508,0.798120
4,다사623056,"MULTIPOLYGON (((127.07547 37.14836, 127.07546 ...",127.076026,37.148813,0.798120
5,다사623053,"MULTIPOLYGON (((127.07548 37.14566, 127.07548 ...",127.076041,37.146109,0.793109
6,다사597048,"MULTIPOLYGON (((127.04623 37.14104, 127.04623 ...",127.046791,37.141494,0.793109
7,다사622053,"MULTIPOLYGON (((127.07435 37.14565, 127.07435 ...",127.074915,37.146105,0.793109
10,다사621058,"MULTIPOLYGON (((127.07320 37.15016, 127.07320 ...",127.073764,37.150608,0.789931
12,다사601085,"MULTIPOLYGON (((127.05054 37.17441, 127.05053 ...",127.051098,37.174862,0.789931
13,다사616037,"MULTIPOLYGON (((127.06768 37.13121, 127.06768 ...",127.068241,37.131658,0.789931
14,다사582108,"MULTIPOLYGON (((127.02901 37.19506, 127.02900 ...",127.029567,37.195511,0.789190


In [83]:
df_end.to_csv('오산시_어린이교통사고_위험지역20개소.csv',encoding='CP949')